## Finetune **Longformer Encoder-Decoder (LED)** on 6K Tokens with Extractive Summarized Data as Input

###In this file we start with the pretraine Longformer LED model and input extractive summaries.

####We use A100 GPU for increase RAM.

We need to upgrade accelerate. Install it on the first session. No need to re-install on the restarted session. Use the pip show to verify the version was installed previously.

In [ ]:
!pip install accelerate -U --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.3/297.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 54.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 56.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 77.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━

Next, we install 🤗Transformers, 🤗Datasets, and `rouge_score`, dill and numpy



In [ ]:
%%capture
!pip install datasets==1.2.1 #do not install again, if installed in previous session
!pip install transformers==4.2.0 #do not install again, if installed in previous session
!pip install rouge_score #do not install again, if installed in previous session

In [ ]:
!pip uninstall -y dill #need 0.3.4, need to reinstall default dill 0.3.8, otherwise it will have both installations, and model will fail
!pip install dill==0.3.4 #after installation restart session AFTER - ALSO reinstall numpy

Found existing installation: dill 0.3.8
Uninstalling dill-0.3.8:
  Successfully uninstalled dill-0.3.8
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 3.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
multiprocess 0.70.16 requires dill>=0.3.8, but you have dill 0.3.4 which is incompatible.


Downgrading numpy version to a previous version that supports np.object. Default numpy installation 1.25.2 causes failure, need to uninstall and install 1.23.5.

In [ ]:
!pip uninstall -y numpy #default numpy installation 1.25.2 causes failure, need to uninstall and install 1.23.5
!pip install numpy==1.23.5 #need to restart session


Found existing installation: numpy 1.25.2
Uninstalling numpy-1.25.2:
  Successfully uninstalled numpy-1.25.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 47.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chex 0.1.86 requires numpy>=1.24.1, but you have numpy 1.23.5 which is incompatible.
pandas-stubs 2.0.3.230814 requires numpy>=1.25.0; python_version >= "3.9", but you have numpy 1.23.5 which is incompatible.


###The next cells are just to check the expected versions installed.

In [ ]:
!pip show accelerate #should show 0.28.0 or 0.29.1

Name: accelerate
Version: 0.29.1
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: zach.mueller@huggingface.co
License: Apache
Location: /usr/local/lib/python3.10/dist-packages
Requires: huggingface-hub, numpy, packaging, psutil, pyyaml, safetensors, torch
Required-by: 


In [ ]:
!pip show datasets #should show 1.2.1

Name: datasets
Version: 1.2.1
Summary: HuggingFace/Datasets is an open library of NLP datasets.
Home-page: https://github.com/huggingface/datasets
Author: HuggingFace Inc.
Author-email: thomas@huggingface.co
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: dill, multiprocess, numpy, pandas, pyarrow, requests, tqdm, xxhash
Required-by: 


In [ ]:
!pip show transformers #should show 4.38.2

Name: transformers
Version: 4.38.2
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: 


In [ ]:
!pip show rouge_score #should show 0.1.2

Name: rouge-score
Version: 0.1.2
Summary: Pure python implementation of ROUGE-1.5.5.
Home-page: https://github.com/google-research/google-research/tree/master/rouge
Author: Google LLC
Author-email: rouge-opensource@google.com
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: absl-py, nltk, numpy, six
Required-by: 


Need dill 0.3.4, need to uninstall default dill 0.3.8, otherwise it will have both installations, and the model will fail. After installation wait to restart session AFTER - ALSO reinstall numpy

In [ ]:
!pip show dill #need 0.3.4 to work, otherwise we get _stack error.


Name: dill
Version: 0.3.4
Summary: serialize all of python
Home-page: https://github.com/uqfoundation/dill
Author: Mike McKerns
Author-email: 
License: 3-clause BSD
Location: /usr/local/lib/python3.10/dist-packages
Requires: 
Required-by: datasets, multiprocess


In [ ]:
!pip show numpy #should show numpy 1.23.5, otherwise it will not work

Name: numpy
Version: 1.23.5
Summary: NumPy is the fundamental package for array computing with Python.
Home-page: https://www.numpy.org
Author: Travis E. Oliphant et al.
Author-email: 
License: BSD
Location: /usr/local/lib/python3.10/dist-packages
Requires: 
Required-by: accelerate, albumentations, altair, arviz, astropy, autograd, blis, bokeh, bqplot, chex, cmdstanpy, contourpy, cufflinks, cupy-cuda12x, cvxpy, datascience, datasets, db-dtypes, dopamine-rl, ecos, flax, folium, geemap, gensim, gym, h5py, holoviews, hyperopt, ibis-framework, imageio, imbalanced-learn, imgaug, jax, jaxlib, librosa, lightgbm, matplotlib, matplotlib-venn, missingno, mizani, ml-dtypes, mlxtend, moviepy, music21, nibabel, numba, numexpr, opencv-contrib-python, opencv-python, opencv-python-headless, opt-einsum, optax, orbax-checkpoint, osqp, pandas, pandas-gbq, pandas-stubs, patsy, plotnine, prophet, pyarrow, pycocotools, pyerfa, pymc, pytensor, python-louvain, PyWavelets, qdldl, qudida, rouge-score, scikit-im

The *Longformer Encoder-Decoder (LED)* was recently added as an extension to [Longformer: The Long-Document Transformer](https://arxiv.org/abs/2004.05150) by Iz Beltagy, Matthew E. Peters, Arman Cohan.

In this notebook we will finetune *LED* for Summarization on MS2 data (https://huggingface.co/datasets/allenai/mslr2022).  LED will be finetuned up to an input length of 8K tokens on a single GPU.

We will leverage 🤗`Seq2SeqTrainer`, gradient checkpointing and as usual 🤗`datasets`.

First, let's check we have a GPU with at least 15GB RAM.

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Apr  7 22:22:18 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0              48W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
import torch
#torch.cuda.empty_cache()

In [ ]:
## crash colab to get more RAM
#!kill -9 -1

In [ ]:
import os
import re
import pandas as pd

In [ ]:
# This cell will authenticate and mount Drive in the Colab.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Load the data

In [ ]:
from tqdm import tqdm


def load_data(file_path):
    chunksize = 5e5 # adjust this value depending on file's size
    df = pd.DataFrame()
    for chunk in tqdm(pd.read_csv(file_path, chunksize=chunksize)):
        df = pd.concat([df, chunk])
    return df

#  path to where you saved the train data
tr_df = load_data('drive/MyDrive/W266_NLP/Project2/summarization/ms2-train-data-plus-extraction.csv') #train data
vl_df = load_data('drive/MyDrive/W266_NLP/Project2/summarization/ms2-val-data-plus-extraction.csv')  #validation data


1it [00:08,  8.60s/it]
1it [00:02,  2.38s/it]


Sanity check the data

In [ ]:
tr_df.head(2)

Unnamed: 0  review_id                                               pmid  \
0           0   30760312  ['22776744', '25271670', '3493740', '1863023',...   
1           1   19588356  ['8532025', '10790348', '17504794', '16793845'...   

                                               title  \
0  ['Improved Cell Survival and Paracrine Capacit...   
1  ['A comparison of continuous intravenous epopr...   

                                            abstract  \
0  ['Although transplantation of adult bone marro...   
1  ['BACKGROUND Primary pulmonary hypertension is...   

                                              target  \
0  Conclusions SC therapy is effective for PAH in...   
1  There was a trend for endothelin receptor anta...   

                                          background  
0  Background Despite significant progress in dru...  
1  BACKGROUND Pulmonary arterial hypertension is ...

In [ ]:
tr_df.abstract.iloc[0]

"['Although transplantation of adult bone marrow mesenchymal stem cells ( BM-MSCs ) holds promise in the treatment for pulmonary arterial hypertension ( PAH ) , the poor survival and differentiation potential of adult BM-MSCs have limited their therapeutic efficiency . Here , we compared the therapeutic efficacy of human embryonic stem cell-derived MSCs ( hESC-MSCs ) with adult BM-MSCs for the treatment of PAH in an animal model . One week following monocrotaline (MCT)-induced PAH , mice were r and omly assigned to receive phosphate-buffered saline ( MCT group ) ; 3.0 × 106 human BM-derived MSCs ( BM-MSCs group ) or 3.0 × 106 hESC-derived MSCs ( hESC-MSCs group ) via tail vein injection . At 3 weeks posttransplantation , the right ventricular systolic pressure ( RVSP ) , degree of RV hypertrophy , and medial wall thickening of pulmonary arteries were lower= , and pulmonary capillary density was higher in the hESC-MSC group as compared with BM-MSC and MCT groups ( all p < 0.05 ) . At 1 

In [ ]:
vl_df.head(2)

Unnamed: 0.1  Unnamed: 0  review_id  \
0             0           0   28514886   
1             1           1   18842808   

                                                pmid  \
0  ['15870317', '20863418', '17991656', '15585783...   
1  ['7872224', '15614200', '8247594', '17161227',...   

                                               title  \
0  ['Quantitative Real-Time PCR Assays To Identif...   
1  ['Effect of short-term ingestion of konjac glu...   

                                            abstract  \
0  ["ABSTRACT A healthy intestinal microbiota is ...   
1  ['The effects of the soluble fiber konjac gluc...   

                                              target  \
0  Current evidence from systematic review and me...   
1  The use of glucomannan did not appear to signi...   

                                          background  \
0  Necrotizing enterocolitis ( NEC ) is one of th...   
1  BACKGROUND Several clinical trials have invest...   

                                 extractive_abstract  
0   ABSTRACT A healthy intestinal microbiota is c...  
1   The subjects were encouraged not to change th...

Lets replace abstract column with the content of extractive abstract, so that it plugs into the rest of the code below, without needing further changes to the code (obviously the target column remains the same):

In [ ]:
tr_df['abstract'] = tr_df['extractive_abstract']

In [ ]:
tr_df.head(2)

Unnamed: 0.1  Unnamed: 0  review_id  \
0             0           0   30760312   
1             1           1   19588356   

                                                pmid  \
0  ['22776744', '25271670', '3493740', '1863023',...   
1  ['8532025', '10790348', '17504794', '16793845'...   

                                               title  \
0  ['Improved Cell Survival and Paracrine Capacit...   
1  ['A comparison of continuous intravenous epopr...   

                                            abstract  \
0   Here , we compared the therapeutic efficacy o...   
1   BACKGROUND Primary pulmonary hypertension is ...   

                                              target  \
0  Conclusions SC therapy is effective for PAH in...   
1  There was a trend for endothelin receptor anta...   

                                          background  \
0  Background Despite significant progress in dru...   
1  BACKGROUND Pulmonary arterial hypertension is ...   

                                 extractive_abstract  
0   Here , we compared the therapeutic efficacy o...  
1   BACKGROUND Primary pulmonary hypertension is ...

In [ ]:
vl_df['abstract'] = vl_df['extractive_abstract']

In [ ]:
vl_df.head(2)

Unnamed: 0.1  Unnamed: 0  review_id  \
0             0           0   28514886   
1             1           1   18842808   

                                                pmid  \
0  ['15870317', '20863418', '17991656', '15585783...   
1  ['7872224', '15614200', '8247594', '17161227',...   

                                               title  \
0  ['Quantitative Real-Time PCR Assays To Identif...   
1  ['Effect of short-term ingestion of konjac glu...   

                                            abstract  \
0   ABSTRACT A healthy intestinal microbiota is c...   
1   The subjects were encouraged not to change th...   

                                              target  \
0  Current evidence from systematic review and me...   
1  The use of glucomannan did not appear to signi...   

                                          background  \
0  Necrotizing enterocolitis ( NEC ) is one of th...   
1  BACKGROUND Several clinical trials have invest...   

                                 extractive_abstract  
0   ABSTRACT A healthy intestinal microbiota is c...  
1   The subjects were encouraged not to change th...

Remove columns we will not use. abstract is our input data, and target is our target data.

In [ ]:
# List of columns to drop
columns_to_drop = ['Unnamed: 0', 'Unnamed: 0.1', 'review_id', 'pmid', 'title', 'background', 'extractive_abstract']

# Drop the unnecessary columns
tr_df = tr_df.drop(columns=columns_to_drop)
#ts_df = ts_df.drop(columns=columns_to_drop)
vl_df = vl_df.drop(columns=columns_to_drop)

tr_df.head(2)


abstract  \
0   Here , we compared the therapeutic efficacy o...   
1   BACKGROUND Primary pulmonary hypertension is ...   

                                              target  
0  Conclusions SC therapy is effective for PAH in...  
1  There was a trend for endothelin receptor anta...

In [ ]:
tr_df.shape , vl_df.shape

((14188, 2), (2021, 2))

the content of the “abstract” column is a single string that happens to look like a list due to its formatting. It’s not a list of strings; it’s just a single string with square brackets and commas included.

In [ ]:
type(tr_df.abstract.iloc[0])

str

To check that we are having enough RAM we can run the following command.
If the randomely allocated GPU is too small, the above cells can be run
to crash the notebook hoping to get a better GPU.

In [ ]:
!nvidia-smi

Sat Apr  6 18:58:30 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0              42W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

Let's start by loading and preprocessing the dataset.



Next, we download the data from the dataframes.

In [ ]:
from datasets import Dataset, load_dataset, load_metric

In [ ]:
# Convert the DataFrame to a Dataset
train_dataset = Dataset.from_pandas(tr_df)
val_dataset = Dataset.from_pandas(vl_df)


In [ ]:
sample_abstract = train_dataset['abstract'][0]
print(sample_abstract)


 Here , we compared the therapeutic efficacy of human embryonic stem cell-derived MSCs ( hESC-MSCs ) with adult BM-MSCs for the treatment of PAH in an animal model . At 1 week posttransplantation , the number of engrafted MSCs in the lungs was found significantly higher in the hESC-MSC group than in the BM-MSC group ( all p < 0.01 ) . At 3 weeks posttransplantation , implanted BM-MSCs were undetectable whereas hESC-MSCs were not only engrafted in injured pulmonary arteries but had also undergone endothelial differentiation . In addition , protein profiling of hESC-MSC- and BM-MSC-conditioned medium revealed a differential paracrine capacity . Classification of these factors into bioprocesses revealed that secreted factors from hESC-MSCs were preferentially involved in early embryonic development and tissue differentiation , especially blood vessel morphogenesis . We concluded that improved cell survival and paracrine capacity of hESC-MSCs provide better therapeutic efficacy than BM-MSC

In [ ]:
tr_df.abstract.iloc[0]

' Here , we compared the therapeutic efficacy of human embryonic stem cell-derived MSCs ( hESC-MSCs ) with adult BM-MSCs for the treatment of PAH in an animal model . At 1 week posttransplantation , the number of engrafted MSCs in the lungs was found significantly higher in the hESC-MSC group than in the BM-MSC group ( all p < 0.01 ) . At 3 weeks posttransplantation , implanted BM-MSCs were undetectable whereas hESC-MSCs were not only engrafted in injured pulmonary arteries but had also undergone endothelial differentiation . In addition , protein profiling of hESC-MSC- and BM-MSC-conditioned medium revealed a differential paracrine capacity . Classification of these factors into bioprocesses revealed that secreted factors from hESC-MSCs were preferentially involved in early embryonic development and tissue differentiation , especially blood vessel morphogenesis . We concluded that improved cell survival and paracrine capacity of hESC-MSCs provide better therapeutic efficacy than BM-MS

Cool! Having downloaded the dataset, let's tokenize it.
We'll import the convenient `AutoTokenizer` class.

##Adding Load Model for Further Training

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("allenai/led-base-16384")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


The max input length is reduced to 6000

In [ ]:
max_input_length = 6000 #tokens, make this longer if we want longer input, like 10,000
max_output_length = 190 #tokens, size of output, this doesnt need to change
batch_size = 2
#batch_size = 4

In [ ]:
def process_data_to_model_inputs(batch):
    # tokenize the inputs and labels
    inputs = tokenizer(
        batch["abstract"],
        padding="max_length",
        truncation=True,
        max_length=max_input_length,
    )
    outputs = tokenizer(
        batch["target"],
        padding="max_length",
        truncation=True,
        max_length=max_output_length,
    )

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask

    # create 0 global_attention_mask lists
    batch["global_attention_mask"] = len(batch["input_ids"]) * [
        [0 for _ in range(len(batch["input_ids"][0]))]
    ]

    # since above lists are references, the following line changes the 0 index for all samples
    batch["global_attention_mask"][0][0] = 1
    batch["labels"] = outputs.input_ids

    # We have to make sure that the PAD token is ignored
    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in labels]
        for labels in batch["labels"]
    ]

    return batch


###Grab a subset of the data, to process a small number, like 100 , then update as such

In [ ]:
train_dataset = train_dataset.select(range(10000))
val_dataset = val_dataset.select(range(50))

Great, having defined the mapping function, let's preprocess the training data

In [ ]:
train_dataset = train_dataset.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=["abstract", "target"],
)

and validation data

In [ ]:
val_dataset = val_dataset.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=["abstract", "target"],
)

Finally, the datasets should be converted into the PyTorch format as follows.

In [ ]:
train_dataset.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "global_attention_mask", "labels"],
)
val_dataset.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "global_attention_mask", "labels"],
)

###Update directory where the model was saved, should be under a sub folder called checkpoint

In [ ]:
from transformers import AutoModelForSeq2SeqLM

In [ ]:
#starting from scratch
led = AutoModelForSeq2SeqLM.from_pretrained("allenai/led-base-16384", gradient_checkpointing=True, use_cache=False)

In [ ]:
# Load the model from the output directory

#from transformers import AutoModelForSeq2SeqLM, AutoConfig

#config = AutoConfig.from_pretrained("/content/drive/MyDrive/W266_NLP/Project2/summarization/e_longformer_8k_plain3/", gradient_checkpointing=True)
#led = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/W266_NLP/Project2/summarization/e_longformer_8k_plain3/", config=config)



In [ ]:
# set generate hyperparameters
led.config.num_beams = 2
led.config.max_length = 190 #was 512, our targets are 190 max tokens at most
led.config.min_length = 26  #was 100, our targets are min 26 tokens
led.config.length_penalty = 2.0
led.config.early_stopping = True
led.config.no_repeat_ngram_size = 3

###Set Evaluation Steps to 250 and Save Steps at 250, Logging_steps at 125, and output directory where it will be saved. Feel free to adjust

In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

In [ ]:
#all files will be stored under folder longformer_plain1, or whatever name the folder its given
#it will automatically save it after each epoch
#output_dir= longformer_plain1
# enable fp16 apex training
#save_steps, and eval_steps depend on batch size
training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    evaluation_strategy="steps",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    fp16=True,
    output_dir="/content/drive/MyDrive/W266_NLP/Project2/summarization/e_ea_longformer_8k_plain1", #where trained model is saved, es is for ext/ab
    logging_steps=250,
    eval_steps=250,
    save_steps=250,
    save_total_limit=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1, #add epochs
)

### rouge metric

In [ ]:
rouge = load_metric("rouge")

In [ ]:
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(
        predictions=pred_str, references=label_str, rouge_types=["rouge1", "rouge2"]
    )

    return {
        "rouge1_precision": round(rouge_output["rouge1"].mid.precision, 4),
        "rouge1_recall": round(rouge_output["rouge1"].mid.recall, 4),
        "rouge1_fmeasure": round(rouge_output["rouge1"].mid.fmeasure, 4),
        "rouge2_precision": round(rouge_output["rouge2"].mid.precision, 4),
        "rouge2_recall": round(rouge_output["rouge2"].mid.recall, 4),
        "rouge2_fmeasure": round(rouge_output["rouge2"].mid.fmeasure, 4),
    }


###Instantiate model

In [ ]:


# Create a new trainer with the loaded model
trainer = Seq2SeqTrainer(
    model=led,
    tokenizer=tokenizer,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)



/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
!nvidia-smi

Sat Apr  6 19:07:40 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0              48W / 400W |   1101MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

###Beging training and Save Trained Model

In [ ]:
# Continue training
trainer.train()
save_to_dir = "/content/drive/MyDrive/W266_NLP/Project2/summarization/e_ea_longformer_8k_plain1"
trainer.save_model(save_to_dir )


Input ids are automatically padded from 6000 to 6144 to be a multiple of `config.attention_window`: 1024


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1339: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 190, 'min_length': 26, 'early_stopping': True, 'num_beams': 2, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3}
Some non-default generation parameters are set in the model config. These should go into a Generati

TrainOutput(global_step=1250, training_loss=3.076843701171875, metrics={'train_runtime': 3516.0194, 'train_samples_per_second': 2.844, 'train_steps_per_second': 0.356, 'total_flos': 3.955378176e+16, 'train_loss': 3.076843701171875, 'epoch': 1.0})